In [1]:
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
import nltk
from nltk.corpus import stopwords

In [3]:
data = load_dataset("sem_eval_2018_task_1", "subtask5.english")
labels = list(data['test'].features.keys())[2:]
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
test_data = data['test'].to_pandas().drop(columns='ID')
train_data = data['train'].to_pandas().drop(columns='ID')

train_labels = train_data.iloc[:,1:].to_numpy()
test_labels = test_data.iloc[:,1:].to_numpy()

# Feature Extraction using TF-IDF
tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))

# Model Training using Logistic Regression
classifier = MultiOutputClassifier(LogisticRegression())

# Creating a pipeline
model = Pipeline([
    ('vectorizer', tfidf),
    ('classifier', classifier)
])

# Training the model
model.fit(train_data['Tweet'], train_labels)

# Model Evaluation
predictions = model.predict(test_data['Tweet'])
print("Accuracy:", metrics.accuracy_score(test_labels, predictions))

Accuracy: 0.17704817428659098


In [5]:
def calc_label_metrics(label, y_targets, y_preds):
    return {
        "label": label,
        "accuracy": metrics.accuracy_score(y_targets, y_preds),
        "precision": metrics.precision_score(y_targets, y_preds, zero_division=0),
        "recall": metrics.recall_score(y_targets, y_preds, zero_division=0),
        "f1": metrics.f1_score(y_targets, y_preds, zero_division=0),
        "mcc": metrics.matthews_corrcoef(y_targets, y_preds),
        "support": y_targets.sum(),
    }


y_probas_all = []
y_targets_all = []
test_loss = 0
correct = 0


sum_precision = 0
sum_recall = 0
sum_f1 = 0
sum_mcc = 0

results = []
for label_index, label in enumerate(labels):
    y_targets, y_preds = test_labels[:, label_index], predictions[:, label_index]
    label_metrics = calc_label_metrics(label, y_targets, y_preds)
    results.append(label_metrics)

    # Sum up metrics for macro-average
    sum_precision += label_metrics["precision"]
    sum_recall += label_metrics["recall"]
    sum_f1 += label_metrics["f1"]
    sum_mcc += label_metrics["mcc"]

# Calculate macro-average metrics
num_labels = len(labels)
macro_avg_precision = sum_precision / num_labels
macro_avg_recall = sum_recall / num_labels
macro_avg_f1 = sum_f1 / num_labels
macro_avg_mcc = sum_mcc / num_labels

# Append macro-average metrics to results
macro_avg_results = {
    "label": "macro_avg",
    "accuracy": None,  # Macro-average accuracy is not typically used
    "precision": macro_avg_precision,
    "recall": macro_avg_recall,
    "f1": macro_avg_f1,
    "mcc": macro_avg_mcc,
    "support": None,  # Support doesn't make sense for macro-average
}
results.append(macro_avg_results)

per_label_results = pd.DataFrame(results, index=[label["label"] for label in results])
display(per_label_results.drop(columns=["label"]).round(3))


,accuracy,precision,recall,f1,mcc,support
anger,0.780,0.796,0.470,0.591,0.484,1101.0
anticipation,0.869,0.375,0.007,0.014,0.036,425.0
disgust,0.758,0.729,0.449,0.555,0.424,1099.0
fear,0.887,0.909,0.268,0.414,0.458,485.0
joy,0.770,0.879,0.556,0.681,0.548,1442.0
love,0.865,0.797,0.198,0.317,0.354,516.0
optimism,0.734,0.754,0.359,0.486,0.378,1143.0
pessimism,0.887,0.818,0.024,0.047,0.128,375.0
sadness,0.781,0.900,0.290,0.438,0.430,960.0
surprise,0.949,1.000,0.029,0.057,0.167,170.0


from matplotlib import pyplot as plt
_df_0['accuracy'].plot(kind='hist', bins=20, title='accuracy')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['precision'].plot(kind='hist', bins=20, title='precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['recall'].plot(kind='hist', bins=20, title='recall')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['f1'].plot(kind='hist', bins=20, title='f1')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='f1', y='mcc', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['accuracy'].plot(kind='line', figsize=(8, 4), title='accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['precision'].plot(kind='line', figsize=(8, 4), title='precision')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['recall'].plot(kind='line', figsize=(8, 4), title='recall')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['f1'].plot(kind='line', figsize=(8, 4), title='f1')
plt.gca().spines[['top', 'right']].set_visible(False)